# Nerevu Group - Technical Interview

Tasks to be completed prior to the technical interview (verbatim):
1. Write a Python script that fetches holiday data and saves it as a csv file named "holidays.csv". To obtain the data, you can either screen scrape the Time and Date site, or parse the API (free trial account required).
2. Using the csv file from (1), write a Python function that returns a list of dictionaries of the next 10 upcoming U.S. holidays and dates.
3. Add a keyword argument "holidayType" to the function from (2) such that it only returns holidays matching the given type.
4. Create a Python powered API that exposes the endpoint, "/holidays". A call to this endpoint should return a JSON response with a list of the next 10 upcoming U.S. holidays and dates.
5. Extend the API from step (4) to optionally take a query parameter "holidayType" and only return holidays matching the given type, e.g., A call to "/holidays?holidayType=federal" should have "Labor Day" as the first returned holiday.


## <a name="TOC"></a> Table of Contents:
---
1. [ Date & Time API ](#S1)
  1. [ Manual Data ](#manual)
  2. [ Data Conversion ](#conversion)
  3. [Holiday Type Cleanup](#type_cleanup)
3. [ Call Upcoming US Holidays ](#S2)
4. [ Filter By Holiday Type ](#S3)


In [1]:
# ------------------------- CONFIGURE ENVIRONMENT ------------------------- #

# Environment hard reset
%reset -f

# Standard math and data libraries
import numpy as np
import pandas as pd

# Plotting libraries
import matplotlib.pyplot as plt
%matplotlib inline

# Libraries for scraping
# import requests
# import urllib.request
# import time
# from bs4 import BeautifulSoup
# import lxml.html as lh

# Date time for date operations
import datetime

# Levenshtein fuzzy comparisons
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process

# Import string cleaning functions
import re

# Flask support
from flask import request, jsonify

# Configure paths
from pathlib import Path
data_path = Path('Datasets')


### <a name="manual"></a> [Manual Data](#TOC)

If I am unable to complete the design of the web scraping system then I can load the data which I copied manually into CSV from the website.


In [2]:
# ------------------------- IMPORT MANUAL DATA ------------------------- #

# Import from CSV
DataFrame = pd.read_csv(data_path / 'ManualData.csv', encoding = "utf-8")

# Replace missing with NaN object
DataFrame.fillna(np.nan);

# See dataframe sample
DataFrame.head(5)


,DATE,WEEKDAY,NAME,TYPE,DETAILS
0,1/1/2019,Tuesday,New Year's Day,Federal Holiday,NaN
1,1/6/2019,Sunday,Epiphany,Christian,NaN
2,1/7/2019,Monday,Orthodox Christmas Day,Orthodox,NaN
3,1/13/2019,Sunday,Stephen Foster Memorial Day,Observance,NaN
4,1/14/2019,Monday,Orthodox New Year,Orthodox,NaN


### <a name="conversion"></a> [Data Conversion](#TOC)

The CSV is best read as a dataframe but we must convert the system into a list of dictionaries as requested. This can be done with a simple command included as a member function of the Pandas DataFrame. For ease of use, we have to convert the `DATE` column into a DateTime variable.


In [3]:
# ------------------------- CONVERT DATA ------------------------- #

# Create Datetime object
Dates = pd.to_datetime( DataFrame[[ DataFrame.columns[0] ]].stack() ).unstack()

# Replace old dates with Datetime
DataFrame[[ DataFrame.columns[0] ]] = Dates[[ Dates.columns[0] ]]

# Convert into a list of dictionaries
DataDict = DataFrame.to_dict('records');

# Inspect a few elements
DataDict[0:2]


[{'DATE': Timestamp('2019-01-01 00:00:00'),
  'WEEKDAY': 'Tuesday',
  'NAME': "New Year's Day",
  'TYPE': 'Federal Holiday',
  'DETAILS': nan},
 {'DATE': Timestamp('2019-01-06 00:00:00'),
  'WEEKDAY': 'Sunday',
  'NAME': 'Epiphany',
  'TYPE': 'Christian',
  'DETAILS': nan}]

### <a name="type_cleanup"></a> [Holiday Type Cleanup](#TOC)

There is an issue with the available holiday types which I ran into later in the API development.


In [4]:
DataFrame[ DataFrame.columns[3] ].unique()

array(['Federal Holiday', 'Christian', 'Orthodox', 'Observance',
       'State holiday', 'Jewish holiday', 'Sporting event',
       'Local observance', 'Observance Christian', 'Hindu Holiday',
       'Clock change/Daylight Saving Time', 'Season', 'Muslim',
       'Jewish commemoration', 'State Government Holiday',
       'Observance Hinduism'], dtype=object)

In [5]:
def shorten( string ):
    return str( string.split()[0] )
    
DataFrame[ DataFrame.columns[3] ] = DataFrame[ DataFrame.columns[3] ].apply(shorten);

## <a name="S2"></a> [§2. Call Upcoming US Holidays](#TOC)
---

The problem we are tasked with in this section follows:

`Using the csv file from (1), write a Python function that returns a list of dictionaries of the next 10 upcoming U.S. holidays and dates.`

In this function I will use the global scope of the CSV data, having loaded it using Pandas in a previous section. There is some discussion as to whether the data should be loaded during the fuction call or loaded under a global scope and used by the function. I will test the performance of each approach.


In [6]:
# ------------------------- CALL UPCOMING US HOLIDAYS ------------------------- #

def NextHolidays1( date_range = 10 ):
    
    # Get current day
    today = datetime.datetime.today()

    # Sort the next dates within the specified range
    next_dates = sorted( [d for d in DataFrame[ DataFrame.columns[0] ].tolist() if d > today], key = lambda s: s - today )[ 0 : date_range ]

    # Select subset from dataframe
    Selection = DataFrame[ DataFrame[ DataFrame.columns[0] ].isin(next_dates) ]
    
    # Convert to dict-array
    return Selection.to_dict('records');


# Call function
NextHolidays1( 2 )


[{'DATE': Timestamp('2019-08-01 00:00:00'),
  'WEEKDAY': 'Thursday',
  'NAME': 'Colorado Day',
  'TYPE': 'Local',
  'DETAILS': 'Colorado'},
 {'DATE': Timestamp('2019-08-04 00:00:00'),
  'WEEKDAY': 'Sunday',
  'NAME': 'Coast Guard Birthday',
  'TYPE': 'Observance',
  'DETAILS': nan}]

This function can call any number of holidays and it updates the current date within the function itself. It is optimized by way of chained methods and lambda functions but it avoids chained indexing with the dataframes as this technique uses two overloaded `__getitem__` calls which drags performance. I have included a `timeit` below to track the performance of the function as well as it's non-Global counter-part.

In [7]:
%timeit NextHolidays1( 1 );

6.23 ms ± 180 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
# ------------------------- LOCALLY SCOPED FUNCTION ------------------------- #

def NextHolidaysLocal( date_range = 10 ):
    
    # Get current day
    today = datetime.datetime.today()
    
    # ---------- Loading Data ---------- #
    
    # Import from CSV
    DataFrame = pd.read_csv(data_path / 'ManualData.csv', encoding = "utf-8")

    # Replace missing with NaN object
    DataFrame.fillna(np.nan);
    
    # Create Datetime object
    Dates = pd.to_datetime( DataFrame[[ DataFrame.columns[0] ]].stack() ).unstack()

    # Replace old dates with Datetime
    DataFrame[[ DataFrame.columns[0] ]] = Dates[[ Dates.columns[0] ]]
    
    # ---------- Find Holidays ---------- #

    # Sort the next dates within the specified range
    next_dates = sorted( [d for d in DataFrame[ DataFrame.columns[0] ].tolist() if d > today], key = lambda s: s - today )[ 0 : date_range ]

    # Select subset from dataframe
    Selection = DataFrame[ DataFrame[ DataFrame.columns[0] ].isin(next_dates) ]
    
    # Convert to dict-array 
    return Selection.to_dict('records');


# Call function
NextHolidaysLocal( 2 )


[{'DATE': Timestamp('2019-08-01 00:00:00'),
  'WEEKDAY': 'Thursday',
  'NAME': 'Colorado Day',
  'TYPE': 'Local observance',
  'DETAILS': 'Colorado'},
 {'DATE': Timestamp('2019-08-04 00:00:00'),
  'WEEKDAY': 'Sunday',
  'NAME': 'Coast Guard Birthday',
  'TYPE': 'Observance',
  'DETAILS': nan}]

In [9]:
%timeit NextHolidaysLocal( 1 );

63.4 ms ± 6.06 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


It looks like the performance of the global scope was superior, unsurprisingly, and by a factor of more than 10. In an API we might have to bite the bullet and use the locally scoped function but I have developed both nonetheless.

## <a name="S3"></a> [§3. Filter By Holiday Type](#TOC)
---

The problem we are tasked with in this section follows:

`Add a keyword argument "holidayType" to the function from (2) such that it only returns holidays matching the given type.`

This is simple when the previous function is treating the holiday data as a dataframe until converting it into an array of dicts, as the dataframe is easier to navigate.


In [10]:
# ------------------------- FILTER BY HOLIDAY TYPE ------------------------- #

def NextHolidays2( date_range = 10, holidayType = None ):
    
    # Get current day
    today = datetime.datetime.today()

    # Sort the next dates within the specified range
    next_dates = sorted( [d for d in DataFrame[ DataFrame.columns[0] ].tolist() if d > today], key = lambda s: s - today )[ 0 : date_range ]

    # Select subset from dataframe
    if holidayType is None:
        Selection = DataFrame[ DataFrame[ DataFrame.columns[0] ].isin(next_dates) ]
    else:
        Selection = DataFrame[ DataFrame[ DataFrame.columns[0] ].isin(next_dates) ].loc[ DataFrame[ DataFrame.columns[3] ] == holidayType ]
    
    # Convert to dict-array
    return Selection.to_dict('records');


# Call function
NextHolidays2( 20, "Federal" )


[{'DATE': Timestamp('2019-09-02 00:00:00'),
  'WEEKDAY': 'Monday',
  'NAME': 'Labor Day',
  'TYPE': 'Federal',
  'DETAILS': nan}]

This is good, but not good enough. Even I had trouble testing it because the capitalization rules are not consistent throughout the holiday type field. What I really need is fuzzy logic to find a good match between user input and determine when the user is truly entering invalid input such as `holidayType = "Captains Log"` versus mismatched input such as `holidayType = "Federal holiday"` when they meant to search for `Federal Holiday`. This will also catch misspellings, which is useful for almost all APIs that involve human interaction in a fast paced business world where spelling is becoming more promiment an issue.

In [11]:
# ------------------------- FUZZY FILTER BY HOLIDAY TYPE ------------------------- #

def NextHolidays2Fuzzy( record_count = 10, holiday_type = None ):
    
    # ---------- Settings / Setup ---------- #

    # Fuzzy logic tolerances
    threshold = 60

    # Get current day
    today = datetime.datetime.today()

    # ---------- Handle date_range ---------- #

    # If num then cast to int for safety
    if record_count: record_count = int(record_count)

    # Sort the next dates within the specified range
    next_dates = sorted( [d for d in DataFrame[ DataFrame.columns[0] ].tolist() if d > today], key = lambda s: s - today )[ 0 : record_count ]

    # ---------- Handle holidayType ---------- #

    # Handle holidayType optionality
    if holiday_type:

        # Pass argument to local variable
        holiday_type = holiday_type

        # Clean numbers and special characters from query
        holiday_type = re.sub('[!@#$%^&*()0123456789<>,./?;:]', '', holiday_type)

        # Setup fuzzy logic comparisons
        similarity = np.array([])
        selected_type = np.nan

        # Iteratively compare the holidayType with the available options
        for option in DataFrame[ DataFrame.columns[3] ].unique():
            similarity = np.append( similarity, fuzz.ratio( holiday_type, option) )

        # Test for acceptable matches
        if max(similarity) >= threshold:
            selected_type = DataFrame[ DataFrame.columns[3] ].unique()[ np.argmax( similarity ) ]

        # Shoutout for debugging
        print("Option: holidayType : {}".format( selected_type ))

        # Run selection
        Selection = DataFrame.loc[ DataFrame[ DataFrame.columns[3] ] == selected_type ].loc[ DataFrame[ DataFrame.columns[0] ] > today ][ 0 : record_count ]

    else:
        # Run selection without holidayType
        Selection = DataFrame[ DataFrame[ DataFrame.columns[0] ].isin(next_dates) ]
        
    # Convert to dict-array then to JSON and return
    return Selection.to_dict('records');


# ---------- DEMONSTRATION ---------- #

# Call function
NextHolidays2Fuzzy( 3, "Federal" )


Option: holidayType : Federal


[{'DATE': Timestamp('2019-09-02 00:00:00'),
  'WEEKDAY': 'Monday',
  'NAME': 'Labor Day',
  'TYPE': 'Federal',
  'DETAILS': nan},
 {'DATE': Timestamp('2019-10-14 00:00:00'),
  'WEEKDAY': 'Monday',
  'NAME': 'Columbus Day',
  'TYPE': 'Federal',
  'DETAILS': 'All except\xa0AK\xa0AR\xa0CA\xa0DE\xa0FL\xa0HI\xa0MI\xa0MN\xa0ND\xa0NM\xa0NV\xa0OR\xa0SD\xa0TX\xa0VT\xa0WA\xa0WY'},
 {'DATE': Timestamp('2019-11-11 00:00:00'),
  'WEEKDAY': 'Monday',
  'NAME': 'Veterans Day',
  'TYPE': 'Federal',
  'DETAILS': nan}]

Lit. This is really cool, I like this a lot. I really wish that programmers regularly anticipated users with my *stile of speling* and used fuzzy-logic to keep the flow steady.

*Written by Austin Dial on 7/25. Edited on 7/26, 7/27, and 7/28.*